# File Exploration

In this notebook I'll explore the other files given in the competition. I don't know if they are gonna be useful, so let's check them out!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import sys
import seaborn as sns
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = (10,8)
plt.style.use('fivethirtyeight')

%matplotlib inline

In [2]:
DATA_PATH = '/home/hugo/Documents/DataScience/Kaggle/kaggle_credit_risk/data/'

In [3]:
files = glob.glob(DATA_PATH + "/*.csv")

dfs = []
for filename in files:
    file = filename.split('/')[-1]#.split('.')[0]
    print(file)

bureau.csv
installments_payments.csv
previous_application.csv
POS_CASH_balance.csv
bureau_balance.csv
application_train.csv
sample_submission.csv
credit_card_balance.csv
application_test.csv
HomeCredit_columns_description.csv


There are a few csvs to look into. 

- bureau.csv
- installments_payments.csv
- previous_application.csv
- POS_CASH_balance.csv
- bureau_balance.csv
- credit_card_balance.csv

Let's start with installments_payments.

# installments_payments.csv

In this dataset we have informations about the installments, such as the number of installments, the amount, and the days in which they suposed to be paid. We can extract some good features regarding delays on payments.

In [4]:
inst_pay = pd.read_csv(DATA_PATH + 'installments_payments.csv')

In [5]:
print('Shape of installments_payments: ' + str(inst_pay.shape))

Shape of installments_payments: (13605401, 8)


In [11]:
inst_pay.head()

SK_ID_PREV  SK_ID_CURR  NUM_INSTALMENT_VERSION  NUM_INSTALMENT_NUMBER  \
0     1054186      161674                     1.0                      6   
1     1330831      151639                     0.0                     34   
2     2085231      193053                     2.0                      1   
3     2452527      199697                     1.0                      3   
4     2714724      167756                     1.0                      2   

   DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  AMT_INSTALMENT  AMT_PAYMENT  \
0          -1180.0             -1187.0        6948.360     6948.360   
1          -2156.0             -2156.0        1716.525     1716.525   
2            -63.0               -63.0       25425.000    25425.000   
3          -2418.0             -2426.0       24350.130    24350.130   
4          -1383.0             -1366.0        2165.040     2160.585   

   PAYMENT_DELAY  OVERPAY  
0            7.0    0.000  
1            0.0    0.000  
2            0.0    0.000  
3            8.0    0.000  
4          -17.0    4.455

In [7]:
inst_pay['PAYMENT_DELAY'] = inst_pay['DAYS_INSTALMENT'] - inst_pay['DAYS_ENTRY_PAYMENT']
inst_pay['OVERPAY'] = inst_pay['AMT_INSTALMENT'] - inst_pay['AMT_PAYMENT']

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


Great! So now let's check how many SD_ID_CURR are repeated in the dataframe.

In [13]:
inst_pay.SK_ID_CURR.value_counts()

145728    372
296205    350
453103    347
189699    344
186851    337
172690    336
418081    332
192083    324
434807    323
217360    318
179215    318
209362    318
380048    317
111137    315
110917    314
422431    313
388717    313
423106    313
382563    312
420239    312
296364    312
288010    311
296729    311
343828    311
410581    310
422662    309
226002    308
363736    306
133713    305
377714    305
         ... 
445528      1
181936      1
109847      1
202223      1
136659      1
191870      1
411181      1
120233      1
365861      1
312953      1
181933      1
402994      1
185615      1
312943      1
249547      1
296860      1
120166      1
183517      1
210233      1
313244      1
189293      1
433263      1
344716      1
247672      1
224970      1
124614      1
248825      1
196631      1
197712      1
438442      1
Name: SK_ID_CURR, Length: 339587, dtype: int64

There is a lot of repeated values. Let's summarize them by using means.

In [17]:
inst_pay = inst_pay.groupby('SK_ID_CURR')[['PAYMENT_DELAY', 'OVERPAY']].mean().reset_index()

## Let's try to merge this information into our main train/test file.

In [30]:
train = pd.read_csv(DATA_PATH + 'application_train.csv')
test = pd.read_csv(DATA_PATH + 'application_test.csv')

In [31]:
train.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

              ...              FLAG_DOCUMENT_18 FLAG_DOCUMENT_19  \
0             ...                             0                0   
1             ...                             0                0   
2             ...                             0                0   
3             ...                             0                0   
4             ...                             0                0   

  FLAG_DOCUMENT_20 FLAG_DOCUMENT_21 AMT_REQ_CREDIT_BUREAU_HOUR  \
0                0                0                        0.0   
1                0                0                        0.0   
2                0                0                        0.0   
3                0                0                        NaN   
4                0                0                        0.0   

  AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_WEEK  \
0                       0.0                         0.0   
1                       0.0                         0.0   
2                       0.0                         0.0   
3                       NaN                         NaN   
4                       0.0                         0.0   

   AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
0                        0.0                        0.0   
1                        0.0                        0.0   
2                        0.0                        0.0   
3                        NaN                        NaN   
4                        0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_YEAR  
0                         1.0  
1                         0.0  
2                         0.0  
3                         NaN  
4                         0.0  

[5 rows x 122 columns]

In order to avoid memory blowout, I'm gonna just merge by SK_ID_CURR

In [32]:
train = train.merge(inst_pay, how = 'left', on = ['SK_ID_CURR'])
test = test.merge(inst_pay, how = 'left', on = ['SK_ID_CURR'])

In [33]:
train.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

      ...      FLAG_DOCUMENT_20 FLAG_DOCUMENT_21 AMT_REQ_CREDIT_BUREAU_HOUR  \
0     ...                     0                0                        0.0   
1     ...                     0                0                        0.0   
2     ...                     0                0                        0.0   
3     ...                     0                0                        NaN   
4     ...                     0                0                        0.0   

  AMT_REQ_CREDIT_BUREAU_DAY AMT_REQ_CREDIT_BUREAU_WEEK  \
0                       0.0                        0.0   
1                       0.0                        0.0   
2                       0.0                        0.0   
3                       NaN                        NaN   
4                       0.0                        0.0   

  AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
0                       0.0                        0.0   
1                       0.0                        0.0   
2                       0.0                        0.0   
3                       NaN                        NaN   
4                       0.0                        0.0   

   AMT_REQ_CREDIT_BUREAU_YEAR  PAYMENT_DELAY     OVERPAY  
0                         1.0      20.421053    0.000000  
1                         0.0       7.160000    0.000000  
2                         0.0       7.666667    0.000000  
3                         NaN      19.375000    0.000000  
4                         0.0       3.636364  452.384318  

[5 rows x 124 columns]

In [36]:
print('Missings in train PAYMENT_DELAY: ' + str(train.PAYMENT_DELAY.isnull().sum()))
print('Missings in train OVERPAY: ' +str(train.OVERPAY.isnull().sum()))

Missings in train PAYMENT_DELAY: 15876
Missings in train OVERPAY: 15876


In [37]:
print('Missings in test PAYMENT_DELAY: ' + str(test.PAYMENT_DELAY.isnull().sum()))
print('Missings in test OVERPAY: ' +str(test.OVERPAY.isnull().sum()))

Missings in test PAYMENT_DELAY: 801
Missings in test OVERPAY: 801


# POS_CASH_balance.csv

In [43]:
pos_cash = pd.read_csv(DATA_PATH + 'POS_CASH_balance.csv')

In [44]:
print('Shape: ' + str(pos_cash.shape))
pos_cash.head()

Shape: (10001358, 8)


SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
0     1803195      182943             -31            48.0   
1     1715348      367990             -33            36.0   
2     1784872      397406             -32            12.0   
3     1903291      269225             -35            48.0   
4     2341044      334279             -35            36.0   

   CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF  
0                   45.0               Active       0           0  
1                   35.0               Active       0           0  
2                    9.0               Active       0           0  
3                   42.0               Active       0           0  
4                   35.0               Active       0           0

In [46]:
pos_cash = pos_cash.groupby('SK_ID_CURR')[['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']].mean().reset_index()

In [47]:
train = train.merge(pos_cash, how = 'left', on = ['SK_ID_CURR'])
test = test.merge(pos_cash, how = 'left', on = ['SK_ID_CURR'])

In [50]:
print('Missings in train CNT_INSTALMENT: ' + str(train.CNT_INSTALMENT.isnull().sum()))
print('Missings in train CNT_INSTALMENT_FUTURE: ' +str(train.CNT_INSTALMENT_FUTURE.isnull().sum()))

Missings in train CNT_INSTALMENT: 18091
Missings in train CNT_INSTALMENT_FUTURE: 18091


In [51]:
print('Missings in test CNT_INSTALMENT: ' + str(test.CNT_INSTALMENT.isnull().sum()))
print('Missings in test CNT_INSTALMENT_FUTURE: ' +str(test.CNT_INSTALMENT_FUTURE.isnull().sum()))

Missings in test CNT_INSTALMENT: 940
Missings in test CNT_INSTALMENT_FUTURE: 940


In [48]:
train.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

           ...            AMT_REQ_CREDIT_BUREAU_HOUR  \
0          ...                                   0.0   
1          ...                                   0.0   
2          ...                                   0.0   
3          ...                                   NaN   
4          ...                                   0.0   

  AMT_REQ_CREDIT_BUREAU_DAY AMT_REQ_CREDIT_BUREAU_WEEK  \
0                       0.0                        0.0   
1                       0.0                        0.0   
2                       0.0                        0.0   
3                       NaN                        NaN   
4                       0.0                        0.0   

  AMT_REQ_CREDIT_BUREAU_MON AMT_REQ_CREDIT_BUREAU_QRT  \
0                       0.0                       0.0   
1                       0.0                       0.0   
2                       0.0                       0.0   
3                       NaN                       NaN   
4                       0.0                       0.0   

  AMT_REQ_CREDIT_BUREAU_YEAR  PAYMENT_DELAY     OVERPAY  CNT_INSTALMENT  \
0                        1.0      20.421053    0.000000       24.000000   
1                        0.0       7.160000    0.000000       10.107143   
2                        0.0       7.666667    0.000000        3.750000   
3                        NaN      19.375000    0.000000       12.000000   
4                        0.0       3.636364  452.384318       15.333333   

   CNT_INSTALMENT_FUTURE  
0              15.000000  
1               5.785714  
2               2.250000  
3               8.650000  
4               8.969697  

[5 rows x 126 columns]

Let's just clean some space in the kernel.

In [52]:
del pos_cash, inst_pay

NameError: name 'pos_cash' is not defined

# credit_card_balance.csv

In [54]:
cc_bal = pd.read_csv(DATA_PATH + 'credit_card_balance.csv')

In [55]:
print(cc_bal.shape)
cc_bal.head()

(3840312, 23)


SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  AMT_BALANCE  \
0     2562384      378907              -6       56.970   
1     2582071      363914              -1    63975.555   
2     1740877      371185              -7    31815.225   
3     1389973      337855              -4   236572.110   
4     1891521      126868              -1   453919.455   

   AMT_CREDIT_LIMIT_ACTUAL  AMT_DRAWINGS_ATM_CURRENT  AMT_DRAWINGS_CURRENT  \
0                   135000                       0.0                 877.5   
1                    45000                    2250.0                2250.0   
2                   450000                       0.0                   0.0   
3                   225000                    2250.0                2250.0   
4                   450000                       0.0               11547.0   

   AMT_DRAWINGS_OTHER_CURRENT  AMT_DRAWINGS_POS_CURRENT  \
0                         0.0                     877.5   
1                         0.0                       0.0   
2                         0.0                       0.0   
3                         0.0                       0.0   
4                         0.0                   11547.0   

   AMT_INST_MIN_REGULARITY     ...      AMT_RECIVABLE  AMT_TOTAL_RECEIVABLE  \
0                 1700.325     ...              0.000                 0.000   
1                 2250.000     ...          64875.555             64875.555   
2                 2250.000     ...          31460.085             31460.085   
3                11795.760     ...         233048.970            233048.970   
4                22924.890     ...         453919.455            453919.455   

   CNT_DRAWINGS_ATM_CURRENT  CNT_DRAWINGS_CURRENT  CNT_DRAWINGS_OTHER_CURRENT  \
0                       0.0                     1                         0.0   
1                       1.0                     1                         0.0   
2                       0.0                     0                         0.0   
3                       1.0                     1                         0.0   
4                       0.0                     1                         0.0   

   CNT_DRAWINGS_POS_CURRENT  CNT_INSTALMENT_MATURE_CUM  NAME_CONTRACT_STATUS  \
0                       1.0                       35.0                Active   
1                       0.0                       69.0                Active   
2                       0.0                       30.0                Active   
3                       0.0                       10.0                Active   
4                       1.0                      101.0                Active   

   SK_DPD  SK_DPD_DEF  
0       0           0  
1       0           0  
2       0           0  
3       0           0  
4       0           0  

[5 rows x 23 columns]

In [59]:
cc_bal = cc_bal.groupby('SK_ID_CURR').mean().drop('SK_ID_PREV',axis=1).reset_index()

In [60]:
train = train.merge(cc_bal, how = 'left', on = ['SK_ID_CURR'])
test = test.merge(cc_bal, how = 'left', on = ['SK_ID_CURR'])

In [61]:
train.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

      ...      AMT_RECEIVABLE_PRINCIPAL AMT_RECIVABLE AMT_TOTAL_RECEIVABLE  \
0     ...                           NaN           NaN                  NaN   
1     ...                           NaN           NaN                  NaN   
2     ...                           NaN           NaN                  NaN   
3     ...                           0.0           0.0                  0.0   
4     ...                           NaN           NaN                  NaN   

  CNT_DRAWINGS_ATM_CURRENT CNT_DRAWINGS_CURRENT CNT_DRAWINGS_OTHER_CURRENT  \
0                      NaN                  NaN                        NaN   
1                      NaN                  NaN                        NaN   
2                      NaN                  NaN                        NaN   
3                      NaN                  0.0                        NaN   
4                      NaN                  NaN                        NaN   

   CNT_DRAWINGS_POS_CURRENT  CNT_INSTALMENT_MATURE_CUM  SK_DPD  SK_DPD_DEF  
0                       NaN                        NaN     NaN         NaN  
1                       NaN                        NaN     NaN         NaN  
2                       NaN                        NaN     NaN         NaN  
3                       NaN                        0.0     0.0         0.0  
4                       NaN                        NaN     NaN         NaN  

[5 rows x 146 columns]

In [62]:

print('Missings in train CNT_INSTALMENT: ' + str(train.AMT_RECEIVABLE_PRINCIPAL.isnull().sum()))
print('Missings in train CNT_INSTALMENT_FUTURE: ' +str(train.AMT_RECEIVABLE_PRINCIPAL.isnull().sum()))

Missings in train CNT_INSTALMENT: 220606
Missings in train CNT_INSTALMENT_FUTURE: 220606


It seems like there is a lot of missing data in those variables. It maybe not a good idea to use them in training.

# Saving data

In [65]:
train.to_csv(DATA_PATH + 'treated_data/train.csv')
test.to_csv(DATA_PATH + 'treated_data/test.csv')